# 対話コーパスの前処理
以降のレクチャーでは、Seq2Seqを使って対話文を生成します。  
宮沢賢治の小説を学習データに使い、賢治風の返事ができるようにモデルを訓練します。  
今回は、そのための準備として、コーパスに前処理を行います。  
小規模なコーパスでも対話文が生成ができるように、漢字は全てひらがなに変換します。

## テキストデータの前処理
今回は、宮沢賢治の小説「銀河鉄道の夜」「セロ弾きのゴーシュ」「注文の多い料理店」などをコーパスとします。  
コーパスが大きほど精度が高まるのですが、学習に時間がかかり過ぎても本コースに支障があるので、10作品に抑えておきます。

In [4]:
import re

novels = ["gingatetsudono_yoru.txt", "serohikino_goshu.txt", "chumonno_oi_ryoriten.txt",
         "gusukobudorino_denki.txt", "kaeruno_gomugutsu.txt", "kaino_hi.txt", "kashiwabayashino_yoru.txt",
         "kazeno_matasaburo.txt", "kiirono_tomato.txt", "oinomorito_zarumori.txt"]  # 青空文庫より

text = ""
for novel in novels:
    with open("../books/kenji_novels/"+novel, mode="r", encoding="utf-8") as f:  # ファイルの読み込み
        text_novel = f.read()
    text_novel = re.sub("《[^》]+》", "", text_novel)  # ルビの削除
    text_novel = re.sub("［[^］]+］", "", text_novel)  # 読みの注意の削除
    text_novel = re.sub("〔[^〕]+〕", "", text_novel)  # 読みの注意の削除
    text_novel = re.sub("[ 　\n「」『』（）｜※＊…]", "", text_novel)  # 全角半角スペース、改行、その他記号の削除
    text += text_novel

print("文字数:", len(text))
# print(text)

文字数: 151453


## 漢字をひらがなに変換
漢字を平仮名に変換し、テキストをひらがな、カタカナ、記号のみにします。  
文字数を絞ることで、モデルの精度を上げて学習時間を短くします。  
漢字を平仮名に変換するためには、pykakasiというライブラリを使います。  
pykakasiがうまく動作しない方は、前処理済みのテキストファイル（kana_kenji.txt）がありますので、それをご利用ください。

In [7]:
from pykakasi import kakasi

seperator = "。"  # 。をセパレータに指定
sentence_list = text.split(seperator)  # セパレーターを使って文章をリストに分割する
sentence_list.pop() # 最後の要素は空の文字列になるので、削除
sentence_list = [x+seperator for x in sentence_list]  # 文章の最後に。を追加

kakasi = kakasi()
kakasi.setMode("J", "H")  # J(漢字) からH(ひらがな)へ
conv = kakasi.getConverter()
for sentence in sentence_list:
    print(sentence)
    print(conv.do(sentence))
    print()

きふきかけていった。

そのうち音はもっとはっきりして来たのだ。
そのうちおとはもっとはっきりしてきたのだ。

ひょろひょろした笛の音も入っていたし、大喇叭のどなり声もきこえた。
ひょろひょろしたふえのおともいっっていたし、だいらっぱのどなりこえもきこえた。

ぼくにはみんなわかって来たのだよ。
ぼくにはみんなわかってきたのだよ。

ネリ、もう少しだよ、しっかり僕につかまっておいで。
ネリ、もうすこしだよ、しっかりぼくにつかまっておいで。

ネリはだまってきれで包んだ小さな卵形の頭を振って、唇を噛んで走った。
ネリはだまってきれでつつんだちいさなたまごがたのあたまをふって、くちびるをかんではしった。

二人がも一度、樺の木の生えた丘をまわったとき、いきなり眼の前に白いほこりのぼやぼや立った大きな道が、横になっているのを見た。
ふたりがもいちど、かばのきのはえたおかをまわったとき、いきなりめのまえにしろいほこりのぼやぼやたったおおきなみちが、よこになっているのをみた。

その右の方から、さっきの音がはっきり聞え、左の方からもう一団り、白いほこりがこっちの方へやって来る。
そのみぎのほうから、さっきのおとがはっきりぶんえ、ひだりのほうからもういちだんり、しろいほこりがこっちのほうへやってくる。

ほこりの中から、チラチラ馬の足が光った。
ほこりのなかから、チラチラうまのあしがひかった。

間もなくそれは近づいたのだ。
かんもなくそれはちかづいたのだ。

ペムペルとネリとは、手をにぎり合って、息をこらしてそれを見た。
ペムペルとネリとは、てをにぎりあって、いきをこらしてそれをみた。

もちろん僕もそれを見た。
もちろんぼくもそれをみた。

やって来たのは七人ばかりの馬乗りなのだ。
やってきたのはしちにんばかりのうまのりなのだ。

馬は汗をかいて黒く光り、鼻からふうふう息をつき、しずかにだくをやっていた。
うまはあせをかいてくろくひかり、はなからふうふういきをつき、しずかにだくをやっていた。

乗ってるものはみな赤シャツで、てかてか光る赤革の長靴をはき、帽子には鷺の毛やなにか、白いひらひらするものをつけていた。
のってるものはみなあかシャツで、てかてかひかるあかがわのながぐつをはき、ぼうしにはさぎのけやなにか、しろいひらひらするものをつけていた。

鬚をはやしたおとなも居れ

エラーが発生しました。  
pykakasiの辞書に無い「苹」という文字が問題となっているようです。  
この文字を予めひらがなに変換した上で、再び変換を行います。

In [ ]:
text = text.replace("苹果", "ひょうか")

seperator = "。"
sentence_list = text.split(seperator) 
sentence_list.pop() 
sentence_list = [x+seperator for x in sentence_list]

for sentence in sentence_list:
    print(sentence)
    print(conv.do(sentence))
    print()

エラーは発生していませんね。  
問題なくひらがなに変換できたようです。  

次に、set()を使って文字の重複を無くし、使用されている文字の一覧を表示してみましょう。

In [8]:
kana_text = conv.do(text)  # 全体をひらがなに変換
print(set(kana_text))  # set()で文字の重複をなくす

/var/folders/nj/7_cg2mvd6x31_3cch6mh5nl40000gn/T/ipykernel_46750/1745145588.py:1: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kana_text = conv.do(text)  # 全体をひらがなに変換
{'々', 'ヴ', 'や', 'ホ', 'ゃ', 'よ', 'ら', 'ア', 'バ', 'ほ', 'グ', 'ノ', 'ぞ', 'ざ', 'ゎ', 'せ', 'あ', 'ん', 'コ', 'ふ', 'テ', 'モ', 'ぐ', 'ワ', 'へ', 'ナ', 'ゲ', 'に', 'ね', 'こ', 'ュ', 'す', 'み', '、', 'ィ', 'オ', 'ぴ', 'づ', 'ろ', 'か', 'ペ', 'め', 'ー', 'ム', 'ひ', 'し', 'ヒ', 'な', 'ま', 'ガ', 'ヤ', 'メ', 'お', 'ブ', 'ぜ', 'ヌ', 'ネ', 'ニ', 'そ', 'ヘ', 'ぶ', 'ソ', 'ぇ', 'プ', 'ぢ', 'る', 'ゅ', 'ル', 'は', 'び', 'ポ', 'ゆ', 'り', 'ッ', 'サ', 'デ', 'ぱ', 'エ', 'ぎ', 'ご', 'と', 'ラ', 'ギ', 'チ', 'ぉ', 'つ', 'ず', 'を', 'む', 'ぁ', 'れ', 'も', 'ケ', 'キ', 'の', 'ン', 'く', 'ど', 'ォ', 'ぺ', 'べ', 'ボ', 'ぃ', 'ス', 'っ', 'ジ', 'ビ', 'げ', 'わ', 'レ', 'が', 'ツ', 'ウ', 'フ', 'セ', 'じ', 'ち', 'パ', 'ゥ', 'ド', 'シ', 'ズ', 'ぬ', 'マ', '・', 'ョ', 'ク', 'ロ', 'て', 'ャ', 'た', 'ぽ', 'ハ', 'い', 'ァ', 'カ', 'ば', 'ミ', '？', 'ぼ', 'リ', '。', 'う', 'ベ', 'ぷ', 'え', 'ピ', 'で', 'ト', 'き', 'さ', 'け

## テキストデータの保存
テキストデータを保存し、いつでも使えるようにします。

In [9]:
print(kana_text)
with open("kana_kenji.txt", mode="w", encoding="utf-8") as f:
    f.write(kana_text)

がいるんだ。にんげんじゃないよ。ぼくたちのほうのだよ。ヘルマンたいさはまっすぐにたってうでをくんでじろじろあたりをめぐっているものをみているねえ、そしてぼくたちのめのしょくでひきょうだったものをすぐみわけるんだ。そしてこら、そのあかげ、いれ。とかういうんだ。そううんわれたらもうおしまいだきょくうずのなかへはいってぐるぐるぐるぐるまわる、なかなかでていいとはうんわないんだ。だからぼくたちそのときはほんとうにきんちょうするよ。けれどもなんにもひきょうをしないものはわりあいへいきだねえ、だいじゅんかんのとちゅうでわざとつかれたとなりのにんのてをはなしたものだのはやくみんなやめるといいとかんがえてきろきろみんなのあしなみをみたりしたものはどれもすっかりいれられちまうんだ。そのうちだんだんぼくらはめぐるだろう。そしてしたのほうにおりるんだ。おしまいはまるでうみとすれすれになる。そのときあちこちのひょうざんに、だいじゅんかんとうちゃくものはこのふきんにおいてすうじつかんきゅうようすべし、きろはかくじんのにんいなるもしょうがいはらいみちにばいするをもてじゅうぶんのかくごをようす。かいようはまさつすくなきもかえってそくどはだいならず。もっともぐどんなるものもっともかしこきものなり、というしろいくいがたっている。これよりせきどうにいたるはっせんろっぴゃくベスターというようなしるべもあちこちにある。だからぼくたちはそのへんでまあごろくにちはやすむねえ、そしてまったくあのへんはおもしろいんだよ。しろくまはいるしね、テッデーベーヤさ。あいつはふざけたやつだねえ、こおりのはじにたってとぼけたかおをしてじっとうみのみずをみているかとおもうとにわかにぜんしであたまをかかえるようにしてね、ざぶんとみずのなかへとびこむんだ。するとからだなかのけがみんなまるでぎんのはりのようにみえるよ。あっぷあっぷおぼれるまねをしたりなんかもするねえ、そんなことをしてふざけながらちゃんとさかなをつかまえるんだからえらいや、さかなをつかまえてこんどはだいいばりでまたこおりにあがるんだ。さかなというものはほんとうにばかなもんだ、ふざけてさえいればだいじょうぶこわくないとおもってるんだ。しろくまはなかなかかしこいよ。それからそのつぎにおもしろいのはほっきょくひかりだよ。ぱちぱちなるんだ、ほんとうになるんだよ。むらさきだのみ

保存したテキストファイルを読み込み、保存できていることを確認します。

In [ ]:
with open("kana_kenji.txt", mode="r", encoding="utf-8") as f:
    kana_text = f.read()
print(kana_text)